# 数据扩增
在赛题中我们需要对的图像进行字符识别，因此需要我们完成的数据的读取操作，同时也需要完成数据扩增（Data Augmentation）操作。

在深度学习中数据扩增方法非常重要
数据扩增可以增加训练集的样本，同时也可以有效缓解模型过拟合的情况，也可以给模型带来的更强的泛化能力。


从颜色空间、尺度空间到样本空间，同时根据不同任务数据扩增都有相应的区别。
对于图像分类，数据扩增一般不会改变标签；对于物体检测，数据扩增会改变物体坐标位置；对于图像分割，数据扩增会改变像素标签。

## 常见的数据扩增方法
在常见的数据扩增方法中，一般会从图像颜色、尺寸、形态、空间和像素等角度进行变换。当然不同的数据扩增方法可以自由进行组合，得到更加丰富的数据扩增方法。

以torchvision为例，常见的数据扩增方法包括：

transforms.CenterCrop 对图片中心进行裁剪
transforms.ColorJitter 对图像颜色的对比度、饱和度和零度进行变换
transforms.FiveCrop 对图像四个角和中心进行裁剪得到五分图像
transforms.Grayscale 对图像进行灰度变换
transforms.Pad 使用固定值进行像素填充
transforms.RandomAffine 随机仿射变换
transforms.RandomCrop 随机区域裁剪
transforms.RandomHorizontalFlip 随机水平翻转
transforms.RandomRotation 随机旋转
transforms.RandomVerticalFlip 随机垂直翻转
赛题任务是需要对图像中的字符进行识别，因此对于字符图片并不能进行翻转操作。

## 常用的数据扩增库
torchvision
https://github.com/pytorch/vision
pytorch官方提供的数据扩增库，提供了基本的数据数据扩增方法，可以无缝与torch进行集成；但数据扩增方法种类较少，且速度中等；

imgaug
https://github.com/aleju/imgaug
imgaug是常用的第三方数据扩增库，提供了多样的数据扩增方法，且组合起来非常方便，速度较快；

albumentations
https://albumentations.readthedocs.io
是常用的第三方数据扩增库，提供了多样的数据扩增方法，对图像分类、语义分割、物体检测和关键点检测都支持，速度较快

# 用Pytorch读取赛题数据
在Pytorch中数据是通过Dataset进行封装，并通过DataLoder进行并行读取。所以我们只需要重载一下数据读取的逻辑就可以完成数据的读取。

In [1]:
import os, sys, glob, shutil, json
import cv2

from PIL import Image
import numpy as np

import torch
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms

class SVHNDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        # 原始SVHN中类别10为数字0
        lbl = np.array(self.img_label[index], dtype=np.int)
        # print(lbl)
        lbl = list(lbl)  + (6 - len(lbl)) * [10]
        return img, torch.from_numpy(np.array(lbl[:6]))

    def __len__(self):
        return len(self.img_path)

train_path = glob.glob('./input/train/*.png')
train_path.sort()
# print(train_path)
train_json = json.load(open('./input/train.json'))
train_label = [train_json[x]['label'] for x in train_json]
print(len(train_path), len(train_label))
# 验证集

val_path = glob.glob('./input/val/*.png')
val_path.sort()
# print(train_path)
val_json = json.load(open('./input/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))

c:\users\wuguo\pycharmprojects\machine-ai\venv\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\wuguo\PycharmProjects\machine-ai\venv\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


30000 30000
10000 10000


如果图中数字为1，9的话，通过以上方法在遍历图片时可以得到[1,9,10,10,10]

In [2]:
train_data = SVHNDataset(train_path, train_label,
          transforms.Compose([
              # 缩放到固定尺寸
              transforms.Resize((64, 128)),

              # 随机颜色变换
              transforms.ColorJitter(0.2, 0.2, 0.2),

              # 加入随机旋转
              transforms.RandomRotation(5),

              # 将图片转换为pytorch 的tesntor
              transforms.ToTensor(),

              # 对图像像素进行归一化
              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]))
val_data = SVHNDataset(val_path, val_label,
          transforms.Compose([
              # 缩放到固定尺寸
              transforms.Resize((64, 128)),

              # 随机颜色变换
              transforms.ColorJitter(0.2, 0.2, 0.2),

              # 加入随机旋转
              transforms.RandomRotation(5),

              # 将图片转换为pytorch 的tesntor
              transforms.ToTensor(),

              # 对图像像素进行归一化
              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]))
#使用DataLoader封装数据
train_loader = torch.utils.data.DataLoader(
       train_data,
    batch_size=10, # 每批样本个数
    shuffle=False, # 是否打乱顺序
    # num_workers=10, # 读取的线程个数
)
val_loader = torch.utils.data.DataLoader(
       val_data,
    batch_size=10, # 每批样本个数
    shuffle=False, # 是否打乱顺序
    # num_workers=10, # 读取的线程个数
)


In [3]:
for batch_idx,(pic,lable) in enumerate(train_loader):
    print(batch_idx)
    print(pic.shape,lable.shape)
    break

0
torch.Size([10, 3, 64, 128]) torch.Size([10, 6])


In [24]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
import torch.nn as nn
from visdom import Visdom
# 定义模型
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()
        # CNN提取特征模块
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        #
        self.fc1 = nn.Linear(32*3*7, 11)
        self.fc2 = nn.Linear(32*3*7, 11)
        self.fc3 = nn.Linear(32*3*7, 11)
        self.fc4 = nn.Linear(32*3*7, 11)
        self.fc5 = nn.Linear(32*3*7, 11)
        self.fc6 = nn.Linear(32*3*7, 11)

    def forward(self, img):
        feat = self.cnn(img)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        c6 = self.fc6(feat)
        return c1, c2, c3, c4, c5, c6

device = torch.device('cuda:0')
model = SVHN_Model1().to(device)
# net = MLP().to(device)

In [25]:
def train(train_loader, model, criterion, optimizer, epoch, global_step):
    # 切换模型为训练模式
    model.train()

    for data in train_loader:
        pic = data[0].to(device)
        c0, c1, c2, c3, c4, c5 = model(pic)
        lables = data[1].long().to(device)
        loss = criterion(c0, lables[:, 0]) + \
                criterion(c1, lables[:, 1]) + \
                criterion(c2, lables[:, 2]) + \
                criterion(c3, lables[:, 3]) + \
                criterion(c4, lables[:, 4]) + \
                criterion(c5, lables[:, 5])
        loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        global_step += 1
        viz.line([loss.item()], [global_step], win='train', update='append')

In [26]:
def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for data in val_loader:
            pic = data[0].to(device)
            c0, c1, c2, c3, c4, c5 = model(pic)
            lables = data[1].long().to(device)
            loss = criterion(c0, lables[:, 0]) + \
                    criterion(c1, lables[:, 1]) + \
                    criterion(c2, lables[:, 2]) + \
                    criterion(c3, lables[:, 3]) + \
                    criterion(c4, lables[:, 4]) + \
                    criterion(c5, lables[:, 5])
            loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [27]:
# 损失函数
criterion = nn.CrossEntropyLoss().to(device)
# 优化器
optimizer = torch.optim.Adam(model.parameters(), 0.005)
# 绘图工具
viz = Visdom()
global_step = 0
viz.line([0.], [0.], win='train',opts=dict(title='train loss.',
                                                   legend=['loss']))
loss_plot = []
# c0_plot = []
# 迭代10个Epoch
best_loss = 1000.0
for epoch in range(10):
    print('Epoch: ', epoch)

    train(train_loader, model, criterion, optimizer, epoch, global_step)
    val_loss = validate(val_loader, model, criterion)
    print(val_loss)
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), './model.pt')

Setting up a new session...


Epoch:  0
0.9168664121627808
Epoch:  1
0.8846571270823479
Epoch:  2
0.894153025329113
Epoch:  3
0.8937509346604348
Epoch:  4
0.8844235015511512
Epoch:  5
0.8827899129986763
Epoch:  6
0.8681970564723015
Epoch:  7
0.8955453754663467
Epoch:  8
0.8790286925435066
Epoch:  9
0.886190665781498


In [28]:
#加载模型的方法
# model.load_state_dict(torch.load(' model.pt'))